# Traitement des photos/videos
TO DO:  
- Récupérer les fichiers qui ont des formats compatibles
- Mettre les noms des dossiers parents dans les descriptions des photo/video
- Verifier les dates des photos/videos (avec les années -> nom du 1er dossier)
- Verifier les formats des photos/videos (avec les extensions compatibles google photos)
- Mettre les photos/videos dans un seul dossier

## Paramètres

In [1]:
dir = "./outputs/Photos_pour_Drive"
output_dir = "./outputs/Photos_traitees"
output_bad_date_file = "./outputs/output_bad_date.txt"
output_bad_format_file = "./outputs/output_bad_format.txt"
output_video_file = "./outputs/output_video.txt"
output_csv_file = "./outputs/output.csv"
output_error_file = "./outputs/output_error.txt"
# extensions compatibles google photos
extensions_photo = ["BMP", "GIF", "HEIC", "ICO", "JPG", "PNG", "TIFF", "WEBP", "RAW", "JPEG"]
extensions_video = ["3GP", "3G2", "ASF", "AVI", "DIVX", "M2T", "M2TS", "M4V", "MKV", "MMV", "MOD", "MOV", "MP4", "MPG", "MTS", "TOD", "WMV"]

## Traitement des photos/videos

In [2]:
import os
import time
from PIL import Image
import piexif
import shutil

In [5]:
def check_format(url_file):
    extension = url_file.split(".")[-1].upper()
    if extension in extensions_photo:
        return "photo"
    elif extension in extensions_video:
        return "video"
    else:
        return "other"

def set_date_photo(url_file, url_new_file, last_date):
    annee = url_file.split(" - ")[0][-4:]
    date_creation = None

    # open file
    img = Image.open(url_file)
    
    # get date creation
    date_creation = img.info.get("date_original")
    if date_creation is None or date_creation == "0000:00:00 00:00:00":
        date_creation = img.info.get("date_digitized")
        if date_creation is None or date_creation == "0000:00:00 00:00:00":
            date_creation = img.info.get("date_created")
            if date_creation == "0000:00:00 00:00:00":
                date_creation = None
    if date_creation is None and os.path.getmtime(url_file) is not None:
        date_creation = os.path.getmtime(url_file)
        if date_creation is None and os.path.getctime(url_file) is not None:
            date_creation = os.path.getctime(url_file)
            if date_creation is None and os.path.getatime(url_file) is not None:
                date_creation = os.path.getatime(url_file)
        if date_creation is not None:
            date_creation = time.strftime("%Y:%m:%d %H:%M:%S", time.gmtime(date_creation))
    # get year of date_creation
    year = date_creation.split(":")[0]
    # CHECK YEAR
    if year != annee:
        print("\t\t\t/!\ BAD YEAR -> " + year + " != " + annee)
        # set date at the last date if it is set
        if last_date.split(":")[0] == annee:
            date_creation = last_date
            print("\t\t\t\tSET TO LAST DATE")
        else:
            date_creation = annee + ":01:01 12:01:01"
            print("\t\t\t\t/!\ NO LAST DATE")

        # write in output_bad_date file
        with open(output_bad_date_file, 'a') as f:
            f.write(url_new_file + "  ->  " + date_creation + "\n")
    
    print("\t\t\tdate_creation: " + date_creation)

    exif_dict = piexif.load(url_file)
    # set date_creation
    exif_dict["0th"][piexif.ImageIFD.DateTime] = date_creation
    exif_dict["Exif"][piexif.ExifIFD.DateTimeOriginal] = date_creation
    exif_dict["Exif"][piexif.ExifIFD.DateTimeDigitized] = date_creation

    return date_creation

In [7]:
set_date_photo("./inputs/photos/.../name.jpg", output_dir, "")

			/!\ BAD YEAR -> 2019 != 1978
				/!\ NO LAST DATE
			date_creation: 1978:01:01 12:01:01


'1978:01:01 12:01:01'

In [ ]:
def save_file(path, album, description = ""):
    # check format
    # check date
    # save file
    # write in output_csv file

In [ ]:
albums = []
# parcour du dossier Photos
for elt in os.listdir(dir):
    albums.append(elt)
# order folder by number asc
albums.sort(key=lambda x: int(x[:4]))

for album in albums:
    print("Album : " + album)
    # parcour du dossier album
    for elt in os.listdir(dir + "/" + album):
        # si c'est un fichier
        if os.path.isfile(dir + "/" + album + "/" + elt):
            # save file
            save_file(dir + "/" + album + "/" + elt, album)
        # si c'est un dossier
        else:
            for elt_elt in os.listdir(dir + "/" + album + "/" + elt):
                # si c'est un fichier
                if os.path.isfile(dir + "/" + album + "/" + elt + "/" + elt_elt):
                    # save file
                    save_file(dir + "/" + album + "/" + elt + "/" + elt_elt, album, elt)

## Verifications & Modifications manuelles
...

In [ ]:
import shutil

In [ ]:
count = 18680
output_dir = "Photos_traitees"
dir = "./inputs/.../dir"
files = os.listdir(dir)
for elt in files:
    shutil.copy(dir + "/" + elt, output_dir + "/" + str(count).zfill(6) + "." + elt.split(".")[-1])
    count += 1

## Upload des photos/videos dans Google Photos
Dans un description appelé "Photos famille" (A créer au préalable)
Nom du dossier + nom de la photo/video = description

Utilisation de l'api de Google Photos (REST API):
```json
{
  "albumId": "AC1QipO2LGKakWY41bgvGoB2irBVkzY51JCjB1uC4s3c",
  "newMediaItems": [
    {
      "description": "###########",
      "simpleMediaItem": {
        "fileName": "###########",
        "uploadToken": "###########"
      }
    }
   , ...
  ]
}
```

### Imports

In [ ]:
import os
import pickle
import requests
import pandas as pd
from Google import Create_Service

directory = "Photos"
output_dir = "Photos_traitees"
output_bad_date_file = "./outputs/output_bad_date.txt"
output_bad_format_file = "./outputs/output_bad_format.txt"
output_video_file = "./outputs/output_video.txt"
output_csv_file = "./outputs/output.csv"
output_error_file = "./outputs/output_error.txt"
# extensions compatibles google photos
extensions_photo = ["BMP", "GIF", "HEIC", "ICO", "JPG", "PNG", "TIFF", "WEBP", "RAW"]
extensions_video = ["3GP", "3G2", "ASF", "AVI", "DIVX", "M2T", "M2TS", "M4V", "MKV", "MMV", "MOD", "MOV", "MP4", "MPG", "MTS", "TOD", "WMV"]

### Preparation google

In [ ]:
CLIENT_SECRET_FILE = 'code_secret_client.json'
API_NAME = 'photoslibrary'
API_VERSION = 'v1'
SCOPES =['https://www.googleapis.com/auth/photoslibrary']

In [ ]:
service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)
print(dir(service))

In [ ]:
# create album
# body = {
#     'album': {
#         'title': "Photos Famille"
#     }
# }
# service.albums().create(body=body).execute()

### Recuperation des données

In [ ]:
df_all = pd.read_csv(output_csv_file)
df_all["description"] = df_all["description"].fillna("")
df_all

In [ ]:
df = df_all[15076:]
df

### Upload sur Google Photos

In [ ]:

# step 1: Upload byte data to Google Server

image_dir = os.path.join(os.getcwd(), output_dir)
upload_url = 'https://photoslibrary.googleapis.com/v1/uploads'
token = pickle.load(open('token_photoslibrary_v1.pickle', 'rb'))

headers = {
    'Authorization': 'Bearer ' + token.token,
    'Content-type': 'application/octet-stream',
    'X-Goog-Upload-Protocol': 'raw'
}

df["response"] = ""
# if type(df["description"]) != string, convert it to string ""
df["description"] = df["description"].astype(str)

# parcour df 40 rows by 40 rows in dataframes names ddf
ddf = pd.DataFrame()
for i in range(0, len(df), 40):
    ddf = df.iloc[i:i+40]
    for index, row in ddf.iterrows():
        image_file = "./" + row["url"]
        image_name = image_file.split("/")[-1]

        headers['X-Goog-Upload-File-Name'] = image_name

        with open(image_file, 'rb') as f:
            img = f.read()
            f.close()
        row["response"] = requests.post(upload_url, data=img, headers=headers)

    print(ddf)

    request_body  = {
        "albumId": "**********************",
        'newMediaItems': [
            {
                'description': row["description"],
                'simpleMediaItem': {
                    'uploadToken': row["response"].content.decode('utf-8')
                }
            } for index, row in ddf.iterrows()
        ]
    }

    upload_response = service.mediaItems().batchCreate(body=request_body).execute()
    print(upload_response)
